In [12]:
import boto3
import pandas as pd
from io import StringIO, BytesIO
from datetime import datetime, timedelta

In [13]:
# Adapter Layer

def read_csv_to_df(bucket, key, decoding = 'utf-8', sep = ','):
    csv_obj = bucket.Object(key=key).get().get('Body').read().decode(decoding)
    data = StringIO(csv_obj)
    df = pd.read_csv(data, delimiter=sep)
    return df

def write_df_to_s3(bucket, df, key):
    out_buffer = BytesIO()
    df.to_parquet(out_buffer, index=False)
    bucket.put_object(Body=out_buffer.getvalue(), Key=key)
    return True

def write_df_to_s3_csv(bucket, df, key):
    out_buffer = StringIO()
    df.to_csv(out_buffer, index=False)
    bucket.put_object(Body=out_buffer.getvalue(), Key=key)
    return True

def list_files_in_prefix(bucket, prefix):
    files = [obj.key for obj in bucket.objects.filter(Prefix=prefix)]
    return files

In [14]:
# Application Layer

def extract(bucket, date_list):
    files = [key for date in date_list for key in list_files_in_prefix(bucket, date)]
    df = pd.concat([read_csv_to_df(bucket, obj) for obj in files], ignore_index=True)
    return df

def transform_report1(df, columns, arg_date):
    df = df.loc[:, columns]
    df.dropna(inplace=True)
    df['opening_price'] = df.sort_values(by=['Time']).groupby(['ISIN', 'Date'])['StartPrice'].transform('first')
    df['closing_price'] = df.sort_values(by=['Time']).groupby(['ISIN', 'Date'])['StartPrice'].transform('last')
    df = df.groupby(['ISIN', 'Date'], as_index=False).agg(opening_price_eur=('opening_price', 'min'), closing_price_eur=('closing_price', 'min'), minimum_price_eur=('MinPrice', 'min'), maximum_price_eur=('MaxPrice', 'max'), daily_traded_volume=('TradedVolume', 'sum'))
    df['prev_closing_price'] = df.sort_values(by=['Date']).groupby(['ISIN'])['closing_price_eur'].shift(1)
    df['change_prev_closing_%'] = (df['closing_price_eur'] - df['prev_closing_price']) / df['prev_closing_price'] * 100
    df.drop(columns=['prev_closing_price'], inplace=True)
    df = df.round(decimals=2)
    df = df[df.Date >= arg_date]
    return df

def load(bucket, df, trg_key, trg_format, meta_key, extract_date_list):
    key = trg_key + datetime.today().strftime("%Y%m%d_%H%M%S") + trg_format
    write_df_to_s3(bucket, df, key)
    update_meta_file(bucket, meta_key, extract_date_list)
    return True

def etl_report1(src_bucket, trg_bucket, date_list, columns, arg_date, trg_key, trg_format, meta_key):
    df = extract(src_bucket, date_list)
    df = transform_report1(df, columns, arg_date)
    extract_date_list = [date for date in date_list if date >= arg_date]
    load(trg_bucket, df, trg_key, trg_format, meta_key, extract_date_list)
    return True
    

In [15]:
# Application Layer - not core

def return_date_list(bucket, arg_date, src_format, meta_key):
    min_date = datetime.strptime(arg_date, src_format).date() - timedelta(days=1)
    today = datetime.today().date()
    try:
        df_meta = read_csv_to_df(bucket, meta_key)
        dates = [(min_date + timedelta(days=x)) for x in range(0, (today-min_date).days + 1)]
        src_dates = set(pd.to_datetime(df_meta['source_date']).dt.date)
        dates_missing = set(dates[1:]) - src_dates
        if dates_missing:
            min_date = min(set(dates[1:]) - src_dates) - timedelta(days=1)
            return_dates = [date.strftime(src_format) for date in dates if date >= min_date]
            return_min_date = (min_date + timedelta(days=1)).strftime(src_format)
        else:
            return_dates = []
            return_min_date = datetime(2200, 1, 1).date()
    except Exception as e:
        print(e)
        return_dates = [(min_date + timedelta(days=x)).strftime(src_format) for x in range(0, (today-min_date).days + 1)]
        return_min_date = arg_date
    return return_min_date, return_dates

def update_meta_file(bucket, meta_key, extract_date_list):
    df_new = pd.DataFrame(columns=['source_date', 'datetime_of_processing'])
    df_new['source_date'] = extract_date_list
    df_new['datetime_of_processing'] = datetime.today().strftime('%Y-%m-%d')
    df_old = read_csv_to_df(bucket, meta_key)
    df_all = pd.concat([df_old, df_new])
    write_df_to_s3_csv(bucket, df_all, meta_key)
    

In [16]:
# main function entrypoint

def main():
    # Parameters/Configurations
    # Later read config
    arg_date = '2022-12-30'
    src_format = '%Y-%m-%d'
    src_bucket = 'xetra-1234'
    trg_bucket = 'xetra-pablo-1234'
    columns = ['ISIN', 'Date', 'Time', 'StartPrice', 'MaxPrice', 'MinPrice', 'EndPrice', 'TradedVolume']
    trg_key = 'xetra_daily_report_'
    trg_format = '.parquet'
    meta_key = 'meta_file.csv'
    
    # Init
    s3 = boto3.resource('s3')
    bucket_src = s3.Bucket(src_bucket)
    bucket_trg = s3.Bucket(trg_bucket)
    
    # run application
    extract_date, date_list = return_date_list(bucket_trg, arg_date, src_format, meta_key)
    etl_report1(bucket_src, bucket_trg, date_list, columns, extract_date, trg_key, trg_format, meta_key)

In [17]:
# run

main()

'source_date'


/tmp/ipykernel_249126/3872885137.py:5: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df = pd.concat([read_csv_to_df(bucket, obj) for obj in files], ignore_index=True)


## Reading the uploaded file

In [18]:
trg_bucket = 'xetra-1234'
s3 = boto3.resource('s3')
bucket_trg = s3.Bucket(trg_bucket)
for obj in bucket_trg.objects.all():
    print(obj.key)

2022-01-03/2022-01-03_BINS_XETR00.csv
2022-01-03/2022-01-03_BINS_XETR01.csv
2022-01-03/2022-01-03_BINS_XETR02.csv
2022-01-03/2022-01-03_BINS_XETR03.csv
2022-01-03/2022-01-03_BINS_XETR04.csv
2022-01-03/2022-01-03_BINS_XETR05.csv
2022-01-03/2022-01-03_BINS_XETR06.csv
2022-01-03/2022-01-03_BINS_XETR07.csv
2022-01-03/2022-01-03_BINS_XETR08.csv
2022-01-03/2022-01-03_BINS_XETR09.csv
2022-01-03/2022-01-03_BINS_XETR10.csv
2022-01-03/2022-01-03_BINS_XETR11.csv
2022-01-03/2022-01-03_BINS_XETR12.csv
2022-01-03/2022-01-03_BINS_XETR13.csv
2022-01-03/2022-01-03_BINS_XETR14.csv
2022-01-03/2022-01-03_BINS_XETR15.csv
2022-01-03/2022-01-03_BINS_XETR16.csv
2022-01-03/2022-01-03_BINS_XETR17.csv
2022-01-03/2022-01-03_BINS_XETR18.csv
2022-01-03/2022-01-03_BINS_XETR19.csv
2022-01-03/2022-01-03_BINS_XETR20.csv
2022-01-03/2022-01-03_BINS_XETR21.csv
2022-01-03/2022-01-03_BINS_XETR22.csv
2022-01-03/2022-01-03_BINS_XETR23.csv
2022-01-04/2022-01-04_BINS_XETR00.csv
2022-01-04/2022-01-04_BINS_XETR01.csv
2022-01-04/2

In [33]:
prq_obj = bucket_trg.Object(key='xetra_daily_report_20210514_133253.parquet').get().get('Body').read()
data = BytesIO(prq_obj)
df_report = pd.read_parquet(data)

In [34]:
df_report

,ISIN,Date,opening_price_eur,closing_price_eur,minimum_price_eur,maximum_price_eur,daily_traded_volume,change_prev_closing_%
0,AT00000FACC2,2021-05-14,8.75,8.81,8.75,8.81,625,0.23
1,AT0000606306,2021-05-14,19.53,19.61,19.50,19.71,3052,0.82
2,AT0000609607,2021-05-14,17.00,16.84,16.82,17.00,490,-1.17
3,AT0000652011,2021-05-14,32.50,32.73,32.50,32.73,869,1.74
4,AT0000720008,2021-05-14,7.00,7.08,7.00,7.08,90,0.28
...,...,...,...,...,...,...,...,...
2612,XS2265368097,2021-05-14,15.18,15.16,15.16,15.18,0,0.23
2613,XS2265369574,2021-05-14,22.57,22.52,22.52,22.57,0,0.64
2614,XS2265369731,2021-05-14,10.19,10.17,10.17,10.19,0,1.57
2615,XS2265370234,2021-05-14,24.11,23.99,23.99,24.11,25,0.66
